# SWIS-100
## Set up run (interactive/single-shot)

In [ ]:
import numpy as np
import pandas as pd
import swis100 as swis


In [ ]:
# Initialise empty DataFrame to collect run configurations
run_configs = pd.DataFrame()

# Initialise empty DataFrame to collect run output stats
run_stats = pd.DataFrame()

In [ ]:
r_id = 'WHBSiOC-V2010-L2015IE+NI-D6-S12'
rc = run_configs

# Note that pandas will tend to default to a float dtype throughout, even when an int is provided; 
# best coerce back to int on use, whenever needed, if important.
# But choosing to make first assignment a boolean value will force pandas dtype to object...
rc.at['use_pyomo', r_id] = False
rc.at['solver_name', r_id] = 'cbc'

rc.at['assumptions_src', r_id] = "SWIS.csv"
rc.at['assumptions_year', r_id] = 2030 # Used to select projected nominal cost 

# https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-usd.en.html
# Change from 7 July 2019 to 8 July 2020
# Minimum (20 March 2020): 1.0707 - Maximum (9 March 2020): 1.1456 - Average: 1.1058
rc.at['usd_to_eur', r_id] = (1.0/1.1058)

# If want test on constant load, set this True
rc.at['constant_load_flag', r_id] = False
if rc.at['constant_load_flag', r_id] :
    rc.at['constant_load (GW)', r_id] = 1
else :
    # Available year(s) for variable IE load data: 2014-2019 inclusive
    rc.at['load_year_start', r_id] = 2014

rc.at['load_scope']="IE+NI" # Only relevant if constant_load_flag=False

rc.at['snapshot_interval', r_id] = 12 # hours

# NB: Hardwired geo-location for renewables_ninja "IE"
# Available year(s) for weather data: solar 1985-2015 inclusive, wind 1980-2016
rc.at['weather_year_start', r_id] = 2010

rc.at['Nyears', r_id] = 6

# Set nominal VRE marginal costs to contol dispatch priority. Lower cost gets higher
# priority). Set all to non-zero so that curtailment/dispatch down at source is 
# preferred over discarding energy from storage (though presumably as long as there are
# *any* fixed costs of storage, this would be avoided by minimising storage size anyway).
rc.at['solar_marginal_cost',r_id] =0.03 # €/MWh
rc.at['onshore_wind_marginal_cost',r_id] =0.02 # €/MWh
rc.at['offshore_wind_marginal_cost',r_id] =0.01 # €/MWh

rc.at['offshore_wind_max_p (GW)', r_id] = +np.inf
#rc.at['offshore_wind_max_p (GW)', r_id] = 20.0

rc.at['onshore_wind_max_p (GW)', r_id] = +np.inf
#rc.at['onshore_wind_max_p (GW)', r_id] = 8.2 
    # 2030 ambition in CAP-2019 (RES-E 70%) - not necessarily upper limit

rc.at['solar_max_p (GW)', r_id] = +np.inf
#rc.at['solar_max_p (GW)', r_id] = 1.5 
    # 2030 ambition in CAP-2019, via NDP (RES-E 55%) - not necessarily upper limit

#rc.at['IC_max_p (GW)', r_id] = +np.inf # Unlimited IC
#rc.at['IC_max_p (GW)', r_id] = 0.5+0.5+0.8 # EWIC + Moyle + Celtic
rc.at['IC_max_p (GW)', r_id] = 0.5+0.5 # EWIC + Moyle
#rc.at['IC_max_p (GW)', r_id] = 0.0 # No IC

rc.at['IC_max_e (TWh)', r_id] = +np.inf

rc.at['Battery_max_p (MW)', r_id] = +np.inf
rc.at['Battery_max_e (MWh)', r_id] = +np.inf

rc.at['H2_electrolysis_tech'] = 'default'
#rc.at['H2_electrolysis_tech'] = 'Nicola-NEL'

rc.at['H2_electrolysis_max_p (GW)', r_id] = +np.inf
rc.at['H2_CCGT_max_p (GW)', r_id] = +np.inf
rc.at['H2_OCGT_max_p (GW)', r_id] = +np.inf

rc.at['H2_storage_tech'] = 'salt cavern'
#rc.at['H2_storage_tech'] = 'rock cavern'
#rc.at['H2_storage_tech'] = 'steel tank'

rc.at['H2_store_max_e (TWh)', r_id] = +np.inf

display(rc)

## Solve the system (interactive/single-shot)

In [ ]:
network = swis.solve_network(run_configs,r_id)

In [ ]:
swis.gather_run_stats(run_configs,run_stats,r_id,network)
with pd.option_context('display.max_rows', None): # Display ALL rows (no ellipsis)
    display(run_stats)

## Some one-off plotting

In [ ]:
Nyears = int(run_configs.loc['Nyears',r_id])
weather_year_start = int(run_configs.loc['weather_year_start',r_id])
weather_year_end = weather_year_start + (Nyears - 1)

plt_start = F"{weather_year_start}-01-01"
plt_stop = F"{weather_year_end}-12-31"

# NB: interactive plotting may be sloooow for large time windows!


In [ ]:
import matplotlib.pyplot as plt
# For non-interactive plots use this magic:
#%matplotlib inline

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 

# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
plt.rcParams["figure.figsize"] = (8,4)

fig, ax = plt.subplots()

rename = {"ic-import" : "import",
          "ic-export" : "export",
          "onshore wind" : "onshore wind",
          "offshore wind" : "offshore wind",
          "solar" : "utility solar PV",
          "battery discharge" : "battery discharge",
          "battery charge" : "battery charge",
          "H2 electrolysis" : "hydrogen electrolysis",
          "H2 CCGT" : "hydrogen CCGT",
          "H2 OCGT" : "hydrogen OCGT"}

rename = {""+k : v for k,v in rename.items()}

rename["local-elec-demand"] = "load"

colors = {"import" : "g",
           "export" : "g",
           "onshore wind" : "royalblue",
           "offshore wind" : "cornflowerblue",
           "utility solar PV" : "y",
           "battery discharge" : "gray",
           "battery charge" : "gray",
           "load" : "k",
           "hydrogen electrolysis" : "m",
           "hydrogen CCGT" : "r",
           "hydrogen OCGT" : "pink"
          }

# KLUDGE: for some reason (solver tolerance?) some solution values that should be strictly 
# positive or negative may be infinitesimally (< 10e-10) of the other sign. This will cause 
# df.plot(stacked=True) to throw an error (requires all values to have same sign, positive or 
# negative). So we do a somewhat ugly ".round(10)" kludge to fix it...

positive = pd.concat((network.generators_t.p,
                      -network.links_t.p1[["ic-import","H2 CCGT","H2 OCGT","battery discharge"]]),
                     axis=1).rename(columns=rename).round(10)
negative = pd.concat((-network.loads_t.p,
                      -network.links_t.p0[["ic-export","H2 electrolysis","battery charge"]]),
                     axis=1).rename(columns=rename).round(10)

print((abs(positive.sum(axis=1) + negative.sum(axis=1)) > 0.1).any())

load_max=network.loads_t.p.sum(axis=1).max()
gen_max=positive.sum(axis=1).max()
demand_max=negative.sum(axis=1).min()

print([load_max,demand_max,gen_max])

# Set y_lim top extra large (*2.0) to make space for the legend...
positive.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                            ylim=(1.2*demand_max, 2.0*gen_max),
                            color=[colors[i] for i in positive.columns])

negative.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                           ylim=(1.2*demand_max, 2.0*gen_max),
                            color=[colors[i] for i in negative.columns])

#ax.set_ylim(1.2*demand_max, 2.0*gen_max)
#ax.set_xlim([plt_start,plt_stop])
ax.set_ylabel("Dispatch (generation is +ve, demand is -ve) [MW]")
ax.legend(ncol=3,loc="upper left")

#fig.tight_layout()
#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

In [ ]:
import matplotlib.pyplot as plt
# For non-interactive plots use this magic:
# %matplotlib inline

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 

plt.rcParams["figure.figsize"] = (8,4)

fig, ax = plt.subplots()

"H2 store underground"
"H2 store tank"
"battery storage"

rename = {"remote-elec-grid-buffer" : "IC buffer", "H2 store" : "H2 store",
    "battery storage" : "battery"}

rename = {""+k : v for k,v in rename.items()}

colors = {"IC buffer" : "m",
        "H2 store" : "b",
        "battery" : "gray"}

storage = network.stores_t.e[["remote-elec-grid-buffer","H2 store",
                              "battery storage"]].rename(columns=rename).round(10)
display(storage)

storage.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                              color=[colors[i] for i in storage.columns])

e_max = storage.sum(axis=1).max()
e_min = storage.sum(axis=1).min()

#ax.set_ylim([demand_max,gen_max])
print([demand_max,gen_max])
ax.set_ylim([0.0, 1.2*e_max])
ax.set_xlim([plt_start,plt_stop])
ax.set_ylabel("Storage [MWh]")
ax.legend(ncol=3,loc="upper left")

fig.tight_layout()

#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

## Dump (volatile) run data to (persistent) .xlsx (just in case...)

**FIXME:** This is very rough and ready - needs much better approach!
Offers no way to re-load/re-run previous configs... :-(


In [ ]:
run_configs.to_excel('runs/2020-11-16/IE-NI-H2store-6Y-12h-run_configs.xlsx')
run_stats.to_excel('runs/2020-11-16/IE-NI-H2store-6Y-12h-run_stats.xlsx')